In [1]:
import torch
import torchvision
import torch.nn as nn
from torchvision import models
import torch.nn.functional as F

In [2]:
# Using downloadable resnet model without all WCL changes to architecture

# replace the first layer with a smaller conv layer [kernel size = 3]
# remove the final fc (linear) layer so the output is of size 2048

class Net(nn.Module): 
    def __init__(self): 
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,    #Dim In: 
                               stride=1, padding=1,     #Dim Out:  
                               bias = False)
        layers = list(models.resnet50(weights=None).children())[1:-1] 
        self.middle = nn.Sequential(*layers)

    def forward(self, x): 
        x = self.conv1(x)
        x = self.middle(x)
        return x.view(x.shape[0], -1)

In [3]:
class ProjectionHead(nn.Module): 
    def __init__(self, dim_in=2048, dim_out=128, dim_hidden=2048): 
        super(ProjectionHead, self).__init__()
        self.linear1 = nn.Linear(dim_in, dim_hidden)
        self.bn1 = nn.BatchNorm1d(dim_hidden)
        self.linear2 = nn.Linear(dim_hidden, dim_hidden)
        self.bn2 = nn.BatchNorm1d(dim_hidden)
        self.linear3 = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(self.bn1(x))
        x = self.linear2(x)
        x = F.relu(self.bn2(x))
        x = self.linear3(x)
        return x    

In [ ]:
class SimCLR(nn.Module): 
    def __init__(self): 
        super(SimCLR, self).__init__()
        self.resnet = Net()
        self.head = ProjectionHead()

    def forward()